# Requirements

In [1]:
!pip install transformers torch datasets huggingface_hub


In [2]:
from huggingface_hub import login

# Paste your token when prompted
login()


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import json
import csv
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import traceback
import sys
import io


# Loading

### Load Model

In [5]:

# === Model Selection ===
model_1_id = "codellama/CodeLlama-7b-Instruct-hf"
#model_1_id = "stabilityai/stablelm-zephyr-3b"


device = "cuda" if torch.cuda.is_available() else "cpu"

# === Load Hugging Face Models ===
def load_model(model_id):
    print(f"Loading model: {model_id}")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto"
    )
    return tokenizer, model

tokenizer1, model1 = load_model(model_1_id)

Loading model: codellama/CodeLlama-7b-Instruct-hf


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

### Load Dataset

In [6]:

n_problems = 10 #
n_samples_per_prompt = 5
print("Loading dataset...")

dataset = load_dataset("dz1/CodeScore-MBPP-ET", split="train")[:n_problems]

print(dataset["test_list"][1])

Loading dataset...


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

MBPP_ET.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/974 [00:00<?, ? examples/s]

['assert similar_elements((3, 4, 5, 6),(5, 7, 4, 10)) == (4, 5)', 'assert similar_elements((1, 2, 3, 4),(5, 4, 3, 7)) == (3, 4)', 'assert similar_elements((11, 12, 14, 13),(17, 15, 14, 13)) == (13, 14)', 'assert similar_elements((7, 1, 6, 7), (7, 2, 5, 7)) == (7,)', 'assert similar_elements((1, 7, 5, 11), (7, 10, 7, 8)) == (7,)', 'assert similar_elements((7, 6, 6, 2), (3, 2, 4, 13)) == (2,)', 'assert similar_elements((3, 1, 6, 9), (3, 7, 6, 8)) == (3, 6)', 'assert similar_elements((8, 5, 4, 9), (7, 3, 8, 7)) == (8,)', 'assert similar_elements((2, 8, 2, 1), (3, 4, 4, 12)) == ()', 'assert similar_elements((3, 9, 9, 3), (4, 11, 6, 14)) == ()', 'assert similar_elements((1, 8, 8, 1), (4, 12, 5, 7)) == ()', 'assert similar_elements((6, 3, 6, 11), (7, 6, 7, 14)) == (6,)', 'assert similar_elements((4, 1, 3, 10), (6, 5, 7, 13)) == ()', 'assert similar_elements((7, 8, 7, 7), (2, 6, 7, 7)) == (7,)', 'assert similar_elements((6, 2, 4, 1), (9, 9, 2, 9)) == (2,)', 'assert similar_elements((2, 2, 5, 

# Generation

### Prompt gen


In [7]:

def extract_cot_and_code(text):
    # Default values
    cot = ""
    code = ""

    # Find the start of the code block
    code_start = text.find("```python")
    code_end = text.find("```", code_start + 1)

    # If code block is found, extract it
    if code_start != -1 and code_end != -1:
        code = text[code_start + len("```python"):code_end].strip()
        cot = text[:code_start].strip()
    else:
        cot = text.strip()  # fallback if no code block

    return cot, code


# === Inference Function ===
def generate_completion(model, tokenizer, prompt, max_new_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()



# === Execute Code Safely ===
def safe_exec(code_str, test_code):
    try:
        global_env = {}
        exec(code_str, global_env)
        exec(test_code, global_env)
        return True
    except Exception as e:
        print("Test failed:", e)
        traceback.print_exc()
        return False



names = ["min_cost", "similar_elements", "is_not_prime", "heap_queue_largest", "count_ways", "differ_At_One_Bit_Pos", "find_char_long", "square_nums", "find_Rotations", "small_nnum"]

def build_prompt_cot(task, i):
    required_name = names[i]
    full_prompt = (
        f"{task['text']}\n\n"
        f"Write a Python function named `{required_name}` that solves the problem.\n"
        f"Use a chain of thought to explain the solution before giving the code.\n\n"
        f"Format your response with:\n"
        f"Chain of Thought:\n<your reasoning here>\n\n"
        f"Python:\n```python\n<your function here>\n```\n"
        f"Output nothing else."
    )
    return full_prompt

def build_prompt_scot(task, i):
    required_name = names[i]
    full_prompt = (
        f"{task['text']}\n\n"
        f"Write a Python function named `{required_name}` that solves the problem.\n"
        f"Use a structured chain of thought to explain the solution before giving the code.\n\n"
        f"Format your response with:\n"
        f"Structured Chain of Thought:\n<your reasoning here>\n\n"
        f"Python:\n```python\n<your function here>\n```\n"
        f"Output nothing else."
    )
    return full_prompt

def build_prompt_cot_self_debug(task, i):
    required_name = names[i]
    full_prompt = (
        f"{task['text']}\n\n"
        f"Write a Python function named `{required_name}` that solves the problem.\n"
        f"Use a chain of thought to explain the solution before giving the code.\n\n"
        f"Before generating code use self debugging to improve your result.\n\n"
        f"Format your response with:\n"
        f"Chain of Thought:\n<your reasoning here>\n\n"
        f"Python:\n```python\n<your function here>\n```\n"
        f"Output nothing else."
    )
    return full_prompt


### Code gen

In [ ]:
results =[]
for i in range(n_problems):
    print(f"\n=== Problem {i+1}: ===")

    task = {
        'text': dataset['text'][i],
        'code': dataset['code'][i],
        'task_id': dataset['task_id'][i],
        'test_setup_code': dataset['test_setup_code'][i],
        'test_list': dataset['test_list'][i],
        'challenge_test_list': dataset['challenge_test_list'][i],
        'entry_point': dataset['entry_point'][i]
    }

    # prompt = build_prompt_cot(task, i) # use for cot
    prompt = build_prompt_scot(task, i) # use for cot and self debug
    full_test_code = task['test_setup_code'] + "\n" + "\n".join(task['test_list'])

    completions = []
    passed_any = False

    for sample_id in range(n_samples_per_prompt):
        print(f"  > Sample {sample_id + 1}/{n_samples_per_prompt}")
        output = generate_completion(model1, tokenizer1, prompt)
        cot, code1 = extract_cot_and_code(output)
        print("======Code=====\n\n"+code1+"\n\n=============")
        passed = safe_exec(code1, full_test_code)

        # Track if any sample passed
        if passed:
            passed_any = True

        completions.append({
            "sample_id": sample_id,
            "cot": cot,
            "code": code1,
            "passed": passed
        })

    results.append({
        "problem": task['entry_point'],
        "task_id": task['task_id'],
        "text": task['text'],
        "prompt": prompt,
        "completions": completions,
        "pass@k": passed_any
    })


# === Summary ===
print("\n=== Pass@k Summary ===")
num_passed = sum(1 for r in results if r["pass@k"])
print(f"pass@{n_samples_per_prompt}: {num_passed}/{n_problems} = {num_passed / n_problems:.2%}")



=== Problem 1: ===
  > Sample 1/5
======Code=====

def min_cost(cost):
    m, n = len(cost), len(cost[0])
    dp = [[float('inf')] * n for _ in range(m)]
    dp[0][0] = cost[0][0]
    for i in range(1, m):
        dp[i][0] = min(dp[i-1][0], dp[i-1][1:] + cost[i][0])
    for j in range(1, n):
        dp[0][j] = min(dp[0][j-1], dp[0][j-1] + cost[0][j])
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1] + cost[i][j])
    return dp[m-1][n-1]

Test failed: min_cost() takes 1 positional argument but 3 were given
  > Sample 2/5


Traceback (most recent call last):
  File "/tmp/ipython-input-209666461.py", line 40, in safe_exec
    exec(test_code, global_env)
  File "<string>", line 2, in <module>
TypeError: min_cost() takes 1 positional argument but 3 were given


======Code=====

def min_cost(cost):
    m, n = len(cost), len(cost[0])
    dp = [[float('inf')] * n for _ in range(m)]
    dp[0][0] = cost[0][0]
    for i in range(1, m):
        dp[i][0] = min(dp[i-1][0], dp[i-1][1:] + cost[i][0])
    for j in range(1, n):
        dp[0][j] = min(dp[0][j-1], dp[0][j-1] + cost[0][j])
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1] + cost[i][j])
    return dp[m-1][n-1]

Test failed: min_cost() takes 1 positional argument but 3 were given
  > Sample 3/5


Traceback (most recent call last):
  File "/tmp/ipython-input-209666461.py", line 40, in safe_exec
    exec(test_code, global_env)
  File "<string>", line 2, in <module>
TypeError: min_cost() takes 1 positional argument but 3 were given


======Code=====

def min_cost(cost):
    # Initialize minimum cost as infinity
    min_cost = float('inf')
    
    # Iterate through each cell in the cost matrix
    for i in range(len(cost)):
        for j in range(len(cost[0])):
            # Calculate the cost of reaching this cell
            current_cost = cost[i][j]
            
            # Iterate backwards through the cost matrix to find the minimum cost path
            for k in range(j-1, -1, -1):
                for l in range(i-1, -1, -1):
                    # Update the minimum cost if a lower cost path is found
                    min_cost = min(min_cost, current_cost + cost[l][k])
    
    # Return the minimum cost
    return min_cost

Test failed: min_cost() takes 1 positional argument but 3 were given
  > Sample 4/5


...

======Code=====

def small_nnum(dataset, n):
    # Step 1: Check if the provided dataset and n are valid
    if not isinstance(dataset, list) or not isinstance(n, int) or n <= 0:
        return "Invalid input"

    # Step 2: Sort the dataset in ascending order
    dataset.sort()

    # Step 3: Get the first n items from the sorted dataset
    smallest_items = dataset[:n]

    # Step 4: Return the smallest n items
    return smallest_items

  > Sample 2/5
======Code=====



Test failed: name 'small_nnum' is not defined
  > Sample 3/5


Traceback (most recent call last):
  File "/tmp/ipython-input-209666461.py", line 40, in safe_exec
    exec(test_code, global_env)
  File "<string>", line 2, in <module>
NameError: name 'small_nnum' is not defined


======Code=====

def small_nnum(dataset, n):
    # Step 1: Check if the input is valid
    if not isinstance(dataset, list) or not isinstance(n, int) or n <= 0:
        return "Invalid input"

    # Step 2: Sort the dataset in ascending order
    dataset.sort()

    # Step 3: Get the first n items from the sorted dataset
    smallest_items = dataset[:n]

    # Step 4: Return the smallest n items
    return smallest_items

  > Sample 4/5
======Code=====



Test failed: name 'small_nnum' is not defined
  > Sample 5/5


Traceback (most recent call last):
  File "/tmp/ipython-input-209666461.py", line 40, in safe_exec
    exec(test_code, global_env)
  File "<string>", line 2, in <module>
NameError: name 'small_nnum' is not defined


======Code=====

def small_nnum(dataset, n):
    # Step 1: Check if the dataset is empty or if n is 0
    if not dataset or n == 0:
        return "Invalid input"

    # Step 2: Sort the dataset in ascending order
    dataset.sort()

    # Step 3: Get the first n items from the sorted dataset
    smallest_items = dataset[:n]

    return smallest_items


=== Pass@k Summary ===
pass@5: 3/10 = 30.00%



### Save to csv

In [ ]:
output_csv = "/content/drive/MyDrive/model_outputs.csv"
fieldnames = ["task_id", "prompt", "problem", "sample_id", "passed", "cot", "code"]

print(results)
# Write to CSV
with open(output_csv, mode="w", newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for r in results:
        for completion in r["completions"]:
            writer.writerow({
                "task_id": r["task_id"],
                "prompt": r["prompt"],
                "problem": r["problem"],
                "sample_id": completion["sample_id"],
                "passed": completion["passed"],
                "cot": completion["cot"].replace("\n", "\\n"),
                "code": completion["code"].replace("\n", "\\n")
            })


[{'problem': 'min_cost', 'task_id': 1, 'text': 'Write a function to find the minimum cost path to reach (m, n) from (0, 0) for the given cost matrix cost[][] and a position (m, n) in cost[][].', 'prompt': 'Write a function to find the minimum cost path to reach (m, n) from (0, 0) for the given cost matrix cost[][] and a position (m, n) in cost[][].\n\nWrite a Python function named `min_cost` that solves the problem.\nUse a structured chain of thought to explain the solution before giving the code.\n\nFormat your response with:\nStructured Chain of Thought:\n<your reasoning here>\n\nPython:\n```python\n<your function here>\n```\nOutput nothing else.', 'completions': [{'sample_id': 0, 'cot': 'Here is the solution for the problem:', 'code': "def min_cost(cost):\n    m, n = len(cost), len(cost[0])\n    dp = [[float('inf')] * n for _ in range(m)]\n    dp[0][0] = cost[0][0]\n    for i in range(1, m):\n        dp[i][0] = min(dp[i-1][0], dp[i-1][1:] + cost[i][0])\n    for j in range(1, n):\n  


----

# Debugging

In [9]:
name = "min_cost"
name = "similar_elements"
if name == "min_cost":
  dataset = load_dataset("dz1/CodeScore-MBPP-ET", split="train")[:1] # load first 2 problems
  num_in=3
  num_out=1
  type_in= "one matrix and two ints"
  type_out = "int"

else:
  dataset = load_dataset("dz1/CodeScore-MBPP-ET", split="train")[1:2] # load first 2 problems
  num_in=2
  num_out=1
  type_in= "tuple of int"
  type_out = "tuple of int"

n_problems = 1

def build_prompt_cot_debugging(task):
    required_name = name
    full_prompt = (
        f"{task['text']}\n\n"
        f"Write a Python function named `{required_name}` that solves the problem.\n"
        f"The function requires {num_in} input parameters of type {type_in} and {num_out} return values of type {type_out}"
        f"Use a chain of thought to explain the solution before giving the code.\n\n"
        f"Format your response with:\n"
        f"Chain of Thought:\n<your reasoning here>\n\n"
        f"Python:\n```python\n<your function here>\n```\n"
        f"Output nothing else."
    )
    return full_prompt



cot =(
    f"0. Receive {num_in} arugments"
    "1. Compute the minimum cost to reach cell (m, n) from (0, 0) in a cost[][] matrix, moving only right or down.\n"
    "2. From any cell (i, j), we can go to (i+1, j) or (i, j+1).\n"
    "3. The cost to reach (i, j) depends on the minimum cost to reach (i-1, j) or (i, j-1).\n"
    "4. Recurrence relation is dp[i][j] = cost[i][j] + min(dp[i-1][j], dp[i][j-1]).\n"
    "5. Base cases is dp[0][0] = cost[0][0]. For first row dp[0][j] = dp[0][j-1] + cost[0][j], first column dp[i][0] = dp[i-1][0] + cost[i][0].\n"
    "6. Use bottom-up DP to fill a 2D dp table based on the recurrence.\n"
    "7. Return dp[m][n] as the final minimum cost.\n"
)
def build_prompt_own_cod_debugging(task):
    required_name = name
    full_prompt = (
        f"{task['text']}\n\n"
        f"Write a Python function named `{required_name}` that solves the problem.\n"
        f"The function requires {num_in} input parameters of type {type_in} and {num_out} return values of type {type_out}\n"
        f"Use the following Chain of Thought to guide you.\n\n"
        f"{cot}\n"
        "Dont forget the required amount of arguments is 3!\n"
        f"Format your response with:\n"
        f"Python:\n```python\n<your function here>\n```\n"
        f"Output nothing else."
    )
    return full_prompt

In [12]:
results = []

print(f"\n=== Problem {1}: ===")

task = {
    'text': dataset['text'][0],
    'code': dataset['code'][0],
    'task_id': dataset['task_id'][0],
    'test_setup_code': dataset['test_setup_code'][0],
    'test_list': dataset['test_list'][0],
    'challenge_test_list': dataset['challenge_test_list'][0],
    'entry_point': dataset['entry_point'][0]
}

# prompt = build_prompt_cot(task, i) # use for cot
prompt = build_prompt_cot_debugging(task) # use for cot and self debug
full_test_code = task['test_setup_code'] + "\n" + "\n".join(task['test_list'])
print("=====prompt====\n\n"+prompt+"\n\n=============")
#print("======test=====\n\n"+full_test_code+"\n\n=============")

completions = []
passed_any = False

for sample_id in range(n_samples_per_prompt):
    print(f"  > Sample {sample_id + 1}/{n_samples_per_prompt}")
    output = generate_completion(model1, tokenizer1, prompt)
    cot, code1 = extract_cot_and_code(output)
    print("======CoT======\n\n"+cot+"\n\n=============")
    print("======Code=====\n\n"+code1+"\n\n=============")
    passed = safe_exec(code1, full_test_code)
    # Track if any sample passed
    if passed:
        passed_any = True

    completions.append({
        "sample_id": sample_id,
        "cot": cot,
        "code": code1,
        "passed": passed
    })

results.append({
    "problem": task['entry_point'],
    "task_id": task['task_id'],
    "text": task['text'],
    "prompt": prompt,
    "completions": completions,
    "pass@k": passed_any
})

# === Summary ===
print("\n=== Pass@k Summary ===")
num_passed = sum(1 for r in results if r["pass@k"])
print(f"pass@{n_samples_per_prompt}: {num_passed}/1 = {num_passed / 1:.2%}")



=== Problem 1: ===
=====prompt====

Write a function to find the similar elements from the given two tuple lists.

Write a Python function named `similar_elements` that solves the problem.
The function requires 2 input parameters of type tuple of int and 1 return values of type tuple of intUse a chain of thought to explain the solution before giving the code.

Format your response with:
Chain of Thought:
<your reasoning here>

Python:
```python
<your function here>
```
Output nothing else.

  > Sample 1/5
======CoT======

---

Chain of Thought:
1. The function should take three parameters: list1, list2, and list3.
2. The function should compare the elements of list1 and list2 to find the similar elements.
3. We can use a nested loop to compare each element of list1 and list2.
4. If a similar element is found, we can add it to list3.
5. Finally, the function should return the list3 which contains the similar elements.

Python:

======Code=====

def similar_elements(list1, list2, list3)

Traceback (most recent call last):
  File "/tmp/ipython-input-209666461.py", line 40, in safe_exec
    exec(test_code, global_env)
  File "<string>", line 2, in <module>
TypeError: similar_elements() missing 1 required positional argument: 'list3'


======CoT======

---

Chain of Thought:
1. The function should compare the elements of the tuple lists.
2. We can use the built-in Python function `zip()` to pair the elements of the two tuple lists.
3. We can use a list comprehension to find the similar elements.
4. We can use the built-in Python function `sorted()` to sort the similar elements.
5. We can return the sorted list of similar elements.

Python:

======Code=====

def similar_elements(list1, list2):
    # Pair the elements of the two tuple lists using zip()
    pairs = zip(list1, list2)
    
    # Use a list comprehension to find the similar elements
    similar = [x for x, y in pairs if x == y]
    
    # Use sorted() to sort the similar elements
    sorted_similar = sorted(similar)
    
    # Return the sorted list of similar elements
    return sorted_similar

Test failed: 
  > Sample 3/5


Traceback (most recent call last):
  File "/tmp/ipython-input-209666461.py", line 40, in safe_exec
    exec(test_code, global_env)
  File "<string>", line 2, in <module>
AssertionError


======CoT======

---

Chain of Thought:
1. The problem is asking to find the similar elements from two tuple lists.
2. To solve this problem, we can use the concept of sets in Python.
3. Sets in Python store multiple items in a single variable.
4. We can convert the tuple lists into sets and then find the intersection of these sets.
5. The intersection of two sets gives us the common elements in both sets.
6. We can return these common elements as a tuple.

Python:

======Code=====

def similar_elements(tuple1, tuple2):
    set1 = set(tuple1)
    set2 = set(tuple2)
    intersection = set1.intersection(set2)
    return tuple(intersection)

  > Sample 4/5
======CoT======

---

Chain of Thought:
1. The problem is asking to find the similar elements from two given tuple lists.
2. To solve this problem, we can use the concept of sets in Python.
3. Sets in Python store multiple items in a single variable.
4. We can convert the tuple lists into sets and find the intersection of these sets.
5.

### To .csv

In [13]:
import csv

output_csv = "/content/drive/MyDrive/model_outputs.csv"
fieldnames = ["task_id", "prompt", "problem", "sample_id", "passed", "cot", "code"]

# Write to CSV
with open(output_csv, mode="w", newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for r in results:
        for completion in r["completions"]:
            writer.writerow({
                "task_id": r["task_id"],
                "prompt": r["prompt"],
                "problem": r["problem"],
                "sample_id": completion["sample_id"],
                "passed": completion["passed"],
                "cot": completion["cot"].replace("\n", "\\n"),
                "code": completion["code"].replace("\n", "\\n")
            })

print(f"\nResults written to: {output_csv}")



Results written to: /content/drive/MyDrive/model_outputs.csv


----
# Innovation

In [14]:
name = "min_cost"
name = "similar_elements"
if name == "min_cost":
  dataset = load_dataset("dz1/CodeScore-MBPP-ET", split="train")[:1] # load first 2 problems
  num_in=3
  num_out=1
  type_in= "one matrix and two ints"
  type_out = "int"

else:
  dataset = load_dataset("dz1/CodeScore-MBPP-ET", split="train")[1:2] # load first 2 problems
  num_in=2
  num_out=1
  type_in= "tuple of int"
  type_out = "tuple of int"

n_problems = 1

In [11]:
def build_prompt_cot_innovation(task):
    required_name = name
    full_prompt = (
        f"{task['text']}\n\n"
        f"Write a Python function named `{required_name}` that solves the problem.\n"
        f"The function requires {num_in} input parameters of type {type_in} and {num_out} return values of type {type_out}\n"
        f"Use a chain of thought to explain the solution before giving the code.\n\n"
        f"Format your response with:\n"
        f"Chain of Thought:\n<your reasoning here>\n\n"
        f"Python:\n```python\n<your function here>\n```\n"
        f"Output nothing else."
    )
    return full_prompt




def run_pass_at_k(task, i, model, tokenizer, k=5, max_retries=1):
    all_runs = []
    passed_count = 0

    for run_id in range(k):
        print(f"\n=== Pass@k Run {run_id + 1}/{k} ===")
        completions, success_attempt, final_prompt = run_problem_with_retry(task, i, model, tokenizer, max_retries=max_retries)

        run_passed = any(comp["passed"] for comp in completions)
        all_runs.append({
            "run_id": run_id,
            "completions": completions,
            "passed": run_passed,
            "successful_attempt": success_attempt,
            "final_prompt": final_prompt
        })

        if run_passed:
            passed_count += 1

    print(f"\nPass@{k}: {passed_count}/{k} = {passed_count / k:.2%}")
    return all_runs, passed_count




def run_problem_with_retry(task, i, model, tokenizer, max_retries=1):
    prompt = build_prompt_cot_innovation(task)
    print("=====prompt====\n\n"+prompt+"\n\n=============")

    original_prompt = prompt  # Save for reference
    full_test_code = task['test_setup_code'] + "\n" + "\n".join(task['test_list'])
    completions = []
    successful_attempt = None

    for attempt in range(max_retries + 1):
        print(f"  > Attempt {attempt + 1}/{max_retries + 1}")

        output = generate_completion(model, tokenizer, prompt)
        cot, code = extract_cot_and_code(output)

        print("====== Code ======\n", code, "\n==================")

        passed, error_msg = safe_exec_with_error(code, full_test_code)

        completions.append({
            "attempt": attempt,
            "cot": cot,
            "code": code,
            "passed": passed,
            "error": error_msg,
            "used_prompt": prompt  # Capture prompt used for this attempt
        })

        if passed:
            print("Passed on attempt", attempt + 1)
            successful_attempt = attempt
            break
        else:
            print("Failed:", error_msg.splitlines()[-1])
            if attempt < max_retries:
                prompt = augment_prompt_with_error(prompt, code, error_msg)

    return completions, successful_attempt, prompt  # Also return final prompt






def safe_exec_with_error(code_str, test_code):
    try:
        global_env = {}
        exec(code_str, global_env)
        exec(test_code, global_env)
        return True, None
    except Exception as e:
        tb = traceback.format_exc()
        return False, tb


def augment_prompt_with_error(prev_prompt, code, error_msg):
    return (
        prev_prompt
        + "\n\n# Note: The previous attempt failed with the following error:\n"
        f"# {error_msg.strip()}\n"
        "# Try to fix the bug based on this feedback.\n"
        "# Previous code:\n"
        f"```python\n{code}\n```\n"
        "# Correct the function below:\n"
    )


In [15]:
results = []

for i in range(n_problems):
    print(f"\n=== Problem {i+1}: ===")

    task = {
        'text': dataset['text'][i],
        'code': dataset['code'][i],
        'task_id': dataset['task_id'][i],
        'test_setup_code': dataset['test_setup_code'][i],
        'test_list': dataset['test_list'][i],
        'challenge_test_list': dataset['challenge_test_list'][i],
        'entry_point': dataset['entry_point'][i]
    }

    all_runs, passed_count = run_pass_at_k(task, i, model1, tokenizer1, k=n_samples_per_prompt, max_retries=1)

    results.append({
        "problem": task['entry_point'],
        "task_id": task['task_id'],
        "text": task['text'],
        "prompt": build_prompt_cot_innovation(task),
        "completions": all_runs,                # includes all retry info per run
        "pass@k": passed_count > 0,             # flag for summary
        "num_passed": passed_count              # number of successful runs
    })

# === Summary ===
print("\n=== Pass@k Summary ===")
num_passed = sum(1 for r in results if r["pass@k"])
print(f"pass@{n_samples_per_prompt}: {num_passed}/{n_problems} = {num_passed / n_problems:.2%}")



=== Problem 1: ===

=== Pass@k Run 1/5 ===
=====prompt====

Write a function to find the similar elements from the given two tuple lists.

Write a Python function named `similar_elements` that solves the problem.
The function requires 2 input parameters of type tuple of int and 1 return values of type tuple of int
Use a chain of thought to explain the solution before giving the code.

Format your response with:
Chain of Thought:
<your reasoning here>

Python:
```python
<your function here>
```
Output nothing else.

  > Attempt 1/2
====== Code ======
 def similar_elements(list1, list2):
    return tuple(set(list1).intersection(list2)) 
Passed on attempt 1

=== Pass@k Run 2/5 ===
=====prompt====

Write a function to find the similar elements from the given two tuple lists.

Write a Python function named `similar_elements` that solves the problem.
The function requires 2 input parameters of type tuple of int and 1 return values of type tuple of int
Use a chain of thought to explain the s

In [13]:
import csv

output_csv = "/content/drive/MyDrive/model_outputs.csv"
fieldnames = [
    "task_id",
    "problem",
    "sample_id",
    "attempt",
    "passed",
    "successful_attempt",
    "cot",
    "code",
    "prompt",
    "final_prompt",
    "error"
]

# Write to CSV
with open(output_csv, mode="w", newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for r in results:
        task_id = r["task_id"]
        problem = r["problem"]
        original_prompt = r["prompt"]

        for run in r["completions"]:  # one run = one sample in pass@k
            run_id = run["run_id"]
            final_prompt = run.get("final_prompt", "")
            successful_attempt = run.get("successful_attempt")

            for comp in run["completions"]:
                writer.writerow({
                    "task_id": task_id,
                    "problem": problem,
                    "sample_id": run_id,
                    "attempt": comp["attempt"],
                    "passed": comp["passed"],
                    "successful_attempt": successful_attempt,
                    "cot": comp["cot"].replace("\n", "\\n"),
                    "code": comp["code"].replace("\n", "\\n"),
                    "prompt": original_prompt.replace("\n", "\\n"),
                    "final_prompt": final_prompt.replace("\n", "\\n"),
                    "error": (comp["error"] or "").replace("\n", "\\n")
                })

print(f"\nResults written to: {output_csv}")



Results written to: /content/drive/MyDrive/model_outputs.csv
